In [1]:
import wandb
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset, load_from_disk, Dataset
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, AutoPeftModelForSequenceClassification
import argparse
import pandas as pd 
import time
import glob
import torch
from torch.utils.data import DataLoader
from accelerate import Accelerator

/home/vs3041/.conda/envs/hf_environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# setting wandb to offline
wandb.init(mode="offline")   




def format_salganik_data(data):

    ## Taking csv and turning it into HF format

    # HF needs a specific form for the dataset
    data["text"] = data["input"]
    data["labels"] = data["output"]

    # keeping only inputs and outputs
    data = data[["text", "labels"]]

    # making sure outputs are 0s and 1s (will need to make this programatic)
    data["labels"] = (data["labels"] == "kid: 1").astype(int)  

    # converting to HF format
    data = Dataset.from_pandas(data)

    return data


def extract_unique_id(filename):

    # extract from books of life 
    if "bol/" in filename:
        start = filename.index('bol/') + len('bol/')
        end = filename.index('.txt')

    elif "outcome/" in filename:
        start = filename.index('outcome/') + len('outcome/')
        end = filename.index('.txt')
    else: 
        ValueError("Can't find unique id from filename")
    
    return filename[start:end]

def format_BOL_data(path_to_training_data):
    
    # Step 1: reading in books of life

    # the books of life are contained in multiple .txt files 
    BOL_txt_files = glob.glob(path_to_training_data + "bol/" + '*.txt')

    books_of_life = []
    unique_ids = []
    data = []

    for txt_file in BOL_txt_files:

        #reading books of life
        with open(txt_file, 'r') as infile:
            book_of_life = infile.read().strip()
            unique_id = extract_unique_id(txt_file)     # extract unique_id from filename

        
        # reading outcomes
        outcome_txt_file = path_to_training_data + "outcome/" + unique_id + '.txt'
        with open(outcome_txt_file, 'r') as infile:
            outcome = infile.read().strip()

        data_for_unique_id = {
            "text": book_of_life,
            "unique_id": unique_id,
            "labels": int(outcome)
        }

        data.append(data_for_unique_id)

    # added with_format("torch")
    data = Dataset.from_list(data)

    return data

def tokenize_and_prepare(data):

    # Tokenizer for HF models

    return tokenizer(data["text"], truncation=True, padding="max_length", max_length=512)

# model arguments from the command line



# # model options
#export model_name="llama3-8b"
#export dataset="bol-temp-1"
#export fine_tune_method="lora"
#export GPU_util="single"
#export params="default"
#export training_folds="0-0"
#export test_folds="0-0"

model_name = "llama3-8b"
dataset = "bol-temp-1"
fine_tune_method = "lora"
GPU_util = "single"
params = "default"
training_folds = "0-0"
first_training_fold, last_training_fold = map(int, training_folds.split("-"))

project_directory = "/scratch/gpfs/vs3041/prefer_prepare/hf_pipeline_MWE/"

fine_tuned_model_name = "-".join([model_name, dataset, fine_tune_method, GPU_util, params, "folds", training_folds])
output_directory = project_directory + "fine_tuned_models/" + fine_tuned_model_name


In [11]:
#### SPECIFIYING MODEL

if model_name == "llama3-8b":
    model_to_read = project_directory + "hf_models/" + "Meta-Llama-3-8B/"
else: 
    raise Exception("Either the model is the wrong place, or we haven't downloaded it yet :(")


#### SPECIFIYING DATA

if dataset == "salganik":
    # reading training data 
    data_to_read = project_directory + "data/salganik_data.csv"
    train_dataset = pd.read_csv(data_to_read)

    # subsetting data to specified training folds 
    train_dataset = train_dataset[train_dataset['fold'].between(first_training_fold, last_training_fold)]

    # formatting the salganik data to get it into a format readable by the LLM
    train_dataset = format_salganik_data(train_dataset)    

    print("using Salganik data")

elif dataset == "bol-temp-1" or dataset == "bol-temp-2":
    data_to_read =  "/scratch/gpfs/vs3041/prefer_prepare/synth/data/e2e/test_template1/train/"

    train_dataset = format_BOL_data(data_to_read)

    print("Using " + dataset)

    print("samples = " + str(len(train_dataset)))

else:
    raise Exception("Dataset not recognised")

#### SPECIFIYING FINE-TUNING METHOD

if fine_tune_method == "full":
    pass
elif fine_tune_method == "lora":
    pass
else: 
    raise Exception("Haven't written code to support other fine-tune methods yet")


#### SPECIFIYING GPU UTILIZATION

if GPU_util == "single":
    pass
else: 
    raise Exception("Haven't written code to support distributed GPU utilization yet")


#### ADDING CUSTOM HYPERPARAMETERS

if params == "default":
    pass
else: 
    raise Exception("Haven't written code to change parameters yet")


Using bol-temp-1
samples = 13642


In [5]:

# might help with CUDA issues: https://blog.gopenai.com/how-to-resolve-runtimeerror-cuda-out-of-memory-d48995452a0
torch.cuda.empty_cache()

# setting up model architecture and initializing tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_to_read, num_labels=2, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_to_read)
tokenizer.pad_token = tokenizer.eos_token 


Loading checkpoint shards: 100%|██████████| 4/4 [01:07<00:00, 16.95s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /scratch/gpfs/vs3041/prefer_prepare/hf_pipeline_MWE/hf_models/Meta-Llama-3-8B/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some parameters are on the meta device device because they were offloaded to the cpu.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
torch.cuda.empty_cache()

# initializing accelerate
accelerator = Accelerator()


if fine_tune_method == "lora":
    peft_config = LoraConfig(task_type="SEQ_CLS", 
                         inference_mode=False, 
                         r=32, 
                         lora_alpha=16, 
                         lora_dropout=0.1)
    peft_model = get_peft_model(model, peft_config)
    print('PEFT Model')
    peft_model.print_trainable_parameters()
else:
    pass


# model hyperparameters
training_args = TrainingArguments(
    output_dir=output_directory,
    logging_steps=20,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    gradient_accumulation_steps=8, # improves memory utilization
    # weight_decay=0.01
    fp16=True,
    gradient_checkpointing=True,
    save_strategy = "no"  # will save model manually usuing 
)

# input padding options 
model.config.pad_token_id = model.config.eos_token_id
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# tokenizing the training data 
tokenized_train_data = train_dataset.map(tokenize_and_prepare, batched=True)


trainer = accelerator.prepare(Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
))

start_time = time.time()

trainer.train()

end_time = time.time()

execution_time = end_time - start_time

print(f"Fine-tuning time: {execution_time} seconds")

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


PEFT Model
trainable params: 13,639,680 || all params: 7,518,572,544 || trainable%: 0.1814


Map: 100%|██████████| 13642/13642 [00:04<00:00, 2742.44 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You shouldn't move a model that is dispatched using accelerate hooks.


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.